# Logs analysis

In [2]:
%matplotlib inline
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import pandas as pd
import re

## List all logs files

In [13]:
import glob
files = glob.glob('../results/logs/**/events.out.tfevents.*', recursive=True)

len(files), files[0:3]

(198,
 ["../results/logs/batch_size=10/'class=RBMCF/visible_size=702/hidden_size=10/regularization=NoRegularization-0.0/learning_rate=ConstantLearningRate-0.01/sampling_method=CD-1/momentum=1/1542223785.0787225/events.out.tfevents.1542223786.paulo-notebook",
  "../results/logs/batch_size=10/'class=RBMCF/visible_size=702/hidden_size=10/regularization=NoRegularization-0.0/learning_rate=ConstantLearningRate-0.01/sampling_method=CD-5/momentum=1/1542224106.1986563/events.out.tfevents.1542224107.paulo-notebook",
  "../results/logs/batch_size=10/'class=RBMCF/visible_size=702/hidden_size=10/regularization=NoRegularization-0.0/learning_rate=ConstantLearningRate-0.1/sampling_method=CD-1/momentum=1/1542226042.397423/events.out.tfevents.1542226044.paulo-notebook"])

## Read a file

In [45]:
file = files[0]

def extract_information(file):
    ea = EventAccumulator(file)
    ea.Reload()
    
    scalars = ea.Tags()['scalars']
    first_scalar = ea.Scalars(scalars[0])
    
    dataframes = pd.DataFrame(first_scalar)[['wall_time', 'step']]
    
    for df_name in ea.Tags()['scalars']:
        df = pd.DataFrame(ea.Scalars(df_name))
        series = df['value'].rename(df_name)
        dataframes = pd.concat([dataframes, series], axis=1)
    
    #train = pd.DataFrame(ea.Scalars('measure/evaluate/train'))
    #validation = pd.DataFrame(ea.Scalars('measure/evaluate/validation'))
    
    del ea

    return dataframes

dfs = extract_information(file)
dfs.head()

#train, validation = extract_information(file)
#train.head()

,wall_time,step,measure/reconstruction/error,measure/activation/min_1,measure/activation/max_1,measure/activation/mean,measure/activation/std,measure/parameters/W/mean,measure/parameters/b_h/mean,measure/parameters/b_v/mean,...,details/measure/evaluate-3/train,details/measure/evaluate-3/validation,details/measure/evaluate-4/train,details/measure/evaluate-4/validation,details/measure/evaluate-5/train,details/measure/evaluate-5/validation,measure/evaluate/RMSE_train,measure/evaluate/RMSE_validation,measure/evaluate/train,measure/evaluate/validation
0,1.542224e+09,1,0.123803,0.0,14.0,5.972839,5.190205,0.002854,0.460400,0.000061,...,0.004938,0.003289,0.013169,0.009868,0.000000,0.003289,0.130439,0.130457,0.004664,0.004386
1,1.542224e+09,21,0.121045,1.0,15.0,5.959671,5.214011,-0.018780,1.553893,-0.000615,...,0.009053,0.006579,0.005761,0.013158,0.000823,0.000000,0.130475,0.130349,0.004115,0.006031
2,1.542224e+09,41,0.118386,1.0,14.0,5.978601,4.750162,-0.025145,1.759731,-0.000235,...,0.016461,0.023026,0.039506,0.046053,0.015638,0.029605,0.129832,0.129551,0.013855,0.018092
3,1.542224e+09,61,0.117291,1.0,15.0,6.077366,4.780845,-0.028178,1.961285,-0.000024,...,0.011523,0.009868,0.013169,0.003289,0.027984,0.026316,0.130015,0.130132,0.011111,0.009320
4,1.542224e+09,81,0.116464,1.0,15.0,5.915226,4.913800,-0.034704,1.956073,-0.000177,...,0.011523,0.006579,0.018107,0.013158,0.024691,0.016447,0.130015,0.130097,0.011111,0.009868


In [50]:
def extract_columns(file):
    regex = r'(.+?)=(.+?)\/'
    test_str = file.replace('../results/logs/', '')
    matches = re.finditer(regex, test_str, re.MULTILINE)

    return {match.groups()[0]: match.groups()[1] for match in matches}

def populate_columns(file, dataframe):
    columns = extract_columns(file)
    
    for k, v in columns.items():
        dataframe[k] = v

    return dataframe

In [51]:
def generate_logs_files(files):
    data = []

    for index, file in enumerate(files):
        print(f'{index+1} of {len(files)}')
        information = extract_information(file)
        information = populate_columns(file, information)
        
        data.append(information)
        
    return pd.concat(data)

data = generate_logs_files(files)

1 of 198
2 of 198
3 of 198
4 of 198
5 of 198
6 of 198
7 of 198
8 of 198
9 of 198
10 of 198
11 of 198
12 of 198
13 of 198
14 of 198
15 of 198
16 of 198
17 of 198
18 of 198
19 of 198
20 of 198
21 of 198
22 of 198
23 of 198
24 of 198
25 of 198
26 of 198
27 of 198
28 of 198
29 of 198
30 of 198
31 of 198
32 of 198
33 of 198
34 of 198
35 of 198
36 of 198
37 of 198
38 of 198
39 of 198
40 of 198
41 of 198
42 of 198
43 of 198
44 of 198
45 of 198
46 of 198
47 of 198
48 of 198
49 of 198
50 of 198
51 of 198
52 of 198
53 of 198
54 of 198
55 of 198
56 of 198
57 of 198
58 of 198
59 of 198
60 of 198
61 of 198
62 of 198
63 of 198
64 of 198
65 of 198
66 of 198
67 of 198
68 of 198
69 of 198
70 of 198
71 of 198
72 of 198
73 of 198
74 of 198
75 of 198
76 of 198
77 of 198
78 of 198
79 of 198
80 of 198
81 of 198
82 of 198
83 of 198
84 of 198
85 of 198
86 of 198
87 of 198
88 of 198
89 of 198
90 of 198
91 of 198
92 of 198
93 of 198
94 of 198
95 of 198
96 of 198
97 of 198
98 of 198
99 of 198
100 of 198
101 of 1

In [52]:
data.head(5)

,wall_time,step,measure/reconstruction/error,measure/activation/min_1,measure/activation/max_1,measure/activation/mean,measure/activation/std,measure/parameters/W/mean,measure/parameters/b_h/mean,measure/parameters/b_v/mean,...,measure/evaluate/train,measure/evaluate/validation,batch_size,'class,visible_size,hidden_size,regularization,learning_rate,sampling_method,momentum
0,1.542224e+09,1,0.123803,0.0,14.0,5.972839,5.190205,0.002854,0.460400,0.000061,...,0.004664,0.004386,10,RBMCF,702,10,NoRegularization-0.0,ConstantLearningRate-0.01,CD-1,1
1,1.542224e+09,21,0.121045,1.0,15.0,5.959671,5.214011,-0.018780,1.553893,-0.000615,...,0.004115,0.006031,10,RBMCF,702,10,NoRegularization-0.0,ConstantLearningRate-0.01,CD-1,1
2,1.542224e+09,41,0.118386,1.0,14.0,5.978601,4.750162,-0.025145,1.759731,-0.000235,...,0.013855,0.018092,10,RBMCF,702,10,NoRegularization-0.0,ConstantLearningRate-0.01,CD-1,1
3,1.542224e+09,61,0.117291,1.0,15.0,6.077366,4.780845,-0.028178,1.961285,-0.000024,...,0.011111,0.009320,10,RBMCF,702,10,NoRegularization-0.0,ConstantLearningRate-0.01,CD-1,1
4,1.542224e+09,81,0.116464,1.0,15.0,5.915226,4.913800,-0.034704,1.956073,-0.000177,...,0.011111,0.009868,10,RBMCF,702,10,NoRegularization-0.0,ConstantLearningRate-0.01,CD-1,1


In [54]:
data.to_csv('extracted_log.csv')